In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import pickle
from tqdm import tqdm
import warnings
import time
import torch

In [8]:
with open('../DeepTimeGeo/trained_models/rb_0711_mask_0/data/training_set.pkl', 'rb') as f:
    loc_np, time_x, poi_x, uid, loc_y, loc_y_exploration, is_commuter, rank_dist, selected_home_rank, selected_work_rank = pickle.load(f)
key_table = pd.read_csv('../DeepTimeGeo/data/coral_gables/0708_r45_100/coral_gables_dict.csv')[['id', 'lon_x', 'lat_x', 'rank']]

In [9]:
loc_np = np.array(loc_np)
uid_np = np.array(uid)

In [43]:
arr_id = np.repeat(uid_np, loc_np.shape[1])
arr_loc = loc_np.flatten()
arr_time = time_x.flatten()*60
arr_day_id = np.tile(np.repeat(np.arange(14), 24), loc_np.shape[0]).flatten()

In [75]:
filled = pd.DataFrame({'id': arr_id, 'rank': arr_loc, 'time': arr_time, 'day': arr_day_id}, dtype='int64')
merged = pd.merge(filled, key_table, on=['id', 'rank'], how='left')
loc_key = merged[['lon_x', 'lat_x']].drop_duplicates().reset_index(drop=True).reset_index()
final = pd.merge(merged, loc_key, on=['lon_x', 'lat_x'], how='left')
final = final.dropna().reset_index(drop=True)

In [92]:
gps = np.array([loc_key['lat_x'], loc_key['lon_x']]).T
np.save('Code/data/ccg/gps.npy', gps)

In [71]:
stay_end = final.groupby(['id','day']).shift(-1).fillna(1440)['time']
final['sta_time'] = stay_end - final['time']
final = final[['id', 'index', 'time', 'sta_time']]
final = final.rename(columns={'id':'usr', 'index': 'loc', 'time': 'tim', 'sta_time': 'sta'})
final.to_csv('Code/data/ccg/TrajSynVAE_input.csv', index=False)

In [90]:
loc_key

,index,lon_x,lat_x
0,0,NaN,NaN
1,1,-80.274,25.776
2,2,-80.305,25.770
3,3,-80.274,25.769
4,4,-80.237,25.779
...,...,...,...
81246,81246,-80.282,25.918
81247,81247,-80.145,25.941
81248,81248,-80.153,25.891
81249,81249,-80.305,25.515
